In [1]:

#!/usr/bin/env python3
"""
Algorithm 1: Static VaR Policy Execution (self-contained demo)

Summary
-------
Given a tabular MDP and a quantile level alpha, this algorithm builds a *static*
policy that maximizes the *Value-at-Risk* (VaR) of discounted returns, per state,
using offline Monte Carlo estimates. At execution time, the policy is fixed (static).

Steps
-----
1) For each state s and action a:
   - Simulate many rollouts that take action a at t=0, then follow a baseline policy.
   - Record the discounted return G per rollout.
   - Compute VaR_alpha over the set of G values => Q_VaR[s, a].
2) Static policy: pi[s] = argmax_a Q_VaR[s, a].
3) Execute pi; it remains fixed (no updates).

CLI
---
# Run the toy demo
python static_var_policy.py --alpha 0.25 --horizon 200 --n-rollouts 1000 --baseline mean_greedy

# Load a CSV domain (optional)
python static_var_policy.py --domain-dir /path/to/domain --alpha 0.25 --horizon 200 --n-rollouts 2000

Expected CSV domain format:
- transitions.csv with columns: s,a,s_next,prob
- rewards.csv with columns: s,a,reward
- domains_info.json with keys: { "gamma": 0.99, "initial_state": 0 }

Outputs
-------
- policy.json: {"alpha": ..., "policy": [...], "Q_VaR": [[...], ...]}
- summary_row.csv: one CSV row labeled "Algorithm 1" with CW, INV1, INV2, MR, POP, RS, GR
"""

import argparse, json
from dataclasses import dataclass
from pathlib import Path
import numpy as np
import pandas as pd

# -------------------- Tabular MDP --------------------
@dataclass
class TabularMDP:
    P: np.ndarray   # shape (S, A, S)
    R: np.ndarray   # shape (S, A)
    gamma: float = 0.99
    initial_state: int = 0

    @property
    def S(self): return self.P.shape[0]
    @property
    def A(self): return self.P.shape[1]

    @staticmethod
    def from_csv(domain_dir: Path, gamma=0.99, initial_state=0):
        transitions = pd.read_csv(domain_dir/"transitions.csv")
        rewards = pd.read_csv(domain_dir/"rewards.csv")
        S = int(max(transitions['s'].max(), transitions['s_next'].max()) + 1)
        A = int(max(transitions['a'].max(), rewards['a'].max()) + 1)
        P = np.zeros((S, A, S), float)
        for _, row in transitions.iterrows():
            P[int(row.s), int(row.a), int(row.s_next)] += float(row.prob)
        P /= P.sum(axis=2, keepdims=True).clip(min=1e-12)
        R = np.zeros((S, A), float)
        for _, row in rewards.iterrows():
            R[int(row.s), int(row.a)] = float(row.reward)
        return TabularMDP(P, R, gamma=float(gamma), initial_state=int(initial_state))

    def step(self, s, a, rng):
        s_next = rng.choice(self.S, p=self.P[s, a])
        r = self.R[s, a]
        return s_next, r

# -------------------- Toy domain --------------------
def load_toy_mdp():
    S, A = 3, 2
    P = np.zeros((S, A, S), float)
    for s in range(S):
        P[s,0,(s+1)%S] = 1.0
    for s in range(S):
        P[s,1,s] = 0.7
        P[s,1,(s+1)%S] = 0.3
    R = np.array([[0.0,-0.1],[0.0,0.0],[1.0,0.2]], float)
    gamma=0.95
    return TabularMDP(P, R, gamma=gamma, initial_state=0)

# -------------------- Risk helpers --------------------
def VaR(samples, alpha=0.25):
    x = np.asarray(samples, float)
    return float(np.quantile(x, alpha))

def CVaR(samples, alpha=0.25):
    x = np.asarray(samples, float)
    q = np.quantile(x, alpha)
    mask = x <= q
    return float(x[mask].mean() if np.any(mask) else q)

def probability_of_profit(x):
    x = np.asarray(x, float)
    return float(np.mean(x > 0))

def downside_deviation(x, mar=0.0):
    x = np.asarray(x, float)
    d = np.minimum(0.0, x - mar)
    return float(np.sqrt(np.mean(d**2)))

def geometric_growth(x):
    y = np.clip(1.0 + np.asarray(x, float), 1e-6, None)
    return float(np.exp(np.mean(np.log(y))) - 1.0)

# -------------------- Baseline policies --------------------
def policy_random(mdp: TabularMDP, s, rng):
    return int(rng.integers(0, mdp.A))

def policy_mean_greedy(mdp: TabularMDP, s, rng=None):
    # Greedy by immediate reward mean R[s,a]
    return int(np.argmax(mdp.R[s]))

BASELINES = {
    "random": policy_random,
    "mean_greedy": policy_mean_greedy,
}

# -------------------- Monte Carlo estimators --------------------
def rollout_with_first_action(mdp: TabularMDP, s0, a0, baseline_name="mean_greedy",
                              horizon=200, seed=0):
    rng = np.random.default_rng(seed)
    baseline = BASELINES[baseline_name]
    s = s0
    G = 0.0
    disc = 1.0

    # first action is fixed
    s, r = mdp.step(s, a0, rng)
    G += disc * r
    disc *= mdp.gamma

    # then follow baseline
    for t in range(1, horizon):
        a = baseline(mdp, s, rng)
        s, r = mdp.step(s, a, rng)
        G += disc * r
        disc *= mdp.gamma
    return G

def estimate_Q_VaR(mdp: TabularMDP, alpha=0.25, n_rollouts=2000, horizon=200,
                   baseline_name="mean_greedy", seed=0):
    rng = np.random.default_rng(seed)
    Q_VaR = np.zeros((mdp.S, mdp.A), float)
    for s in range(mdp.S):
        for a in range(mdp.A):
            rets = [rollout_with_first_action(mdp, s, a, baseline_name, horizon, seed=int(rng.integers(0, 2**31-1)))
                    for _ in range(n_rollouts)]
            Q_VaR[s, a] = VaR(rets, alpha)
    return Q_VaR

def build_static_var_policy(mdp: TabularMDP, alpha=0.25, n_rollouts=2000, horizon=200,
                            baseline_name="mean_greedy", seed=0):
    Qv = estimate_Q_VaR(mdp, alpha=alpha, n_rollouts=n_rollouts, horizon=horizon,
                        baseline_name=baseline_name, seed=seed)
    pi = np.argmax(Qv, axis=1).astype(int)
    return Qv, pi

def evaluate_policy(mdp: TabularMDP, policy, horizon=200, n_episodes=2000, seed=123):
    rng = np.random.default_rng(seed)
    rets = []
    for ep in range(n_episodes):
        s = mdp.initial_state
        disc = 1.0
        G = 0.0
        for t in range(horizon):
            a = int(policy[s])
            s, r = mdp.step(s, a, rng)
            G += disc * r
            disc *= mdp.gamma
        rets.append(G)
    rets = np.asarray(rets, float)
    return {
        "mean": float(np.mean(rets)),
        "std": float(np.std(rets)),
        "min": float(np.min(rets)),
        "max": float(np.max(rets)),
        "returns": rets,
    }

def summarize_row_from_returns(returns, alpha=0.25):
    r = np.asarray(returns, float)
    mu = float(np.mean(r))
    sd = float(np.std(r))
    # Placeholders for INV1/INV2/CW/MR/etc. Replace with your exact definitions as needed.
    CW   = VaR(r, alpha)            # VaR as a cost-like proxy
    INV1 = mu + sd                  # placeholder
    INV2 = mu + 2*sd                # placeholder
    MR   = mu                       # mean return
    POP  = probability_of_profit(r)
    RS   = downside_deviation(r)    # placeholder risk score
    GR   = geometric_growth(r)
    return dict(CW=CW, INV1=INV1, INV2=INV2, MR=MR, POP=POP, RS=RS, GR=GR)

# -------------------- CLI --------------------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--domain-dir", type=str, default=None, help="Folder with transitions.csv, rewards.csv, domains_info.json")
    ap.add_argument("--alpha", type=float, default=0.25)
    ap.add_argument("--n-rollouts", type=int, default=1000)
    ap.add_argument("--horizon", type=int, default=200)
    ap.add_argument("--baseline", type=str, default="mean_greedy", choices=list(BASELINES.keys()))
    ap.add_argument("--seed", type=int, default=0)
    ap.add_argument("--outdir", type=str, default="algorithm1_static_var_out")
    args = ap.parse_args()

    outdir = Path(args.outdir); outdir.mkdir(parents=True, exist_ok=True)

    if args.domain_dir:
        # Load CSV domain
        info_path = Path(args.domain_dir)/"domains_info.json"
        gamma, s0 = 0.99, 0
        if info_path.exists():
            meta = json.load(open(info_path, "r"))
            gamma = float(meta.get("gamma", gamma)); s0 = int(meta.get("initial_state", s0))
        mdp = TabularMDP.from_csv(Path(args.domain_dir), gamma=gamma, initial_state=s0)
    else:
        mdp = load_toy_mdp()

    Qv, pi = build_static_var_policy(mdp, alpha=args.alpha, n_rollouts=args.n_rollouts,
                                     horizon=args.horizon, baseline_name=args.baseline, seed=args.seed)

    # Save policy
    with open(outdir/"policy.json", "w") as f:
        json.dump({"alpha": args.alpha, "policy": pi.tolist(), "Q_VaR": Qv.tolist()}, f, indent=2)

    # Evaluate and summarize as "Algorithm 1" row
    eval_res = evaluate_policy(mdp, pi, horizon=args.horizon, n_episodes=max(500, args.n_rollouts//2), seed=123)
    row = summarize_row_from_returns(eval_res["returns"], alpha=args.alpha)
    row_df = pd.DataFrame([row], index=["Algorithm 1"], columns=["CW","INV1","INV2","MR","POP","RS","GR"])
    row_df.to_csv(outdir/"summary_row.csv", float_format="%.4f")

    print("Saved ->", (outdir/"policy.json").resolve())
    print("Saved ->", (outdir/"summary_row.csv").resolve())

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--domain-dir DOMAIN_DIR] [--alpha ALPHA] [--n-rollouts N_ROLLOUTS]
                             [--horizon HORIZON] [--baseline {random,mean_greedy}] [--seed SEED] [--outdir OUTDIR]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Khalid\AppData\Roaming\jupyter\runtime\kernel-08b03dfa-005b-418f-a294-56349476e8d9.json


SystemExit: 2

C:\Users\Khalid\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ---------- 1) Your results table ----------
rows = [
    ("q¯d",        -9.11, 237.19, 970.08,  -2.79, -14348.60, 50.0, 4.78),
    ("Algorithm 1",-9.11, 237.05, 968.31,  -2.84, -14348.60, 50.0, 4.78),
    ("qd",         -9.11, 236.88, 967.60,  -2.85, -14348.60, 50.0, 4.78),
    ("nVaR",       -86.10,202.49, 953.00, -20.00, -14348.60, 50.0, 0.00),
    ("dVaR",       -9.20, 234.75, 953.00, -18.21, -14348.60, 50.0, 0.00),
    ("E",          -9.72, 234.60, 957.45,  -2.95, -15077.83, 32.8, 3.14),
    ("CVaR",       -9.72, 232.31, 956.57,  -4.12, -14348.60, 50.0, 3.14),
    ("EVaR",       -9.72, 234.98, 957.45,  -2.95, -14348.60, 50.0, 2.82),
]
columns = ["CW","INV1","INV2","MR","POP","RS","GR"]
df = pd.DataFrame([r[1:] for r in rows], index=[r[0] for r in rows], columns=columns)

# ---------- 2) Objective directions ----------
OBJECTIVE = {"CW": -1, "INV1": 1, "INV2": 1, "MR": 1, "POP": 1, "RS": -1, "GR": 1}
use_cols = [c for c in df.columns if OBJECTIVE.get(c, 0) != 0]

# ---------- 3a) Rank matrix ----------
rank_mat = pd.DataFrame(index=df.index, columns=use_cols, dtype=float)
for c in use_cols:
    rank_mat[c] = df[c].rank(ascending=(OBJECTIVE[c] == 1)==False, method="min")

# ---------- 3b) Min–Max normalized scores ----------
def minmax_scores(col, direction):
    x = df[col].astype(float).values
    if direction == -1:
        x = -x
    x_min, x_max = np.min(x), np.max(x)
    return (x - x_min) / (x_max - x_min) if x_max > x_min else np.ones_like(x) * 0.5

norm_scores = pd.DataFrame({c: minmax_scores(c, OBJECTIVE[c]) for c in use_cols}, index=df.index)

# ---------- 3c) Z-scores ----------
z_mat = pd.DataFrame(index=df.index, columns=use_cols, dtype=float)
for c in use_cols:
    x = df[c].astype(float).values
    if OBJECTIVE[c] == -1:
        x = -x
    mu, sd = float(np.mean(x)), float(np.std(x))
    z_mat[c] = (x - mu) / sd if sd > 0 else np.zeros_like(x)

# ---------- 3d) Pairwise wins ----------
methods = df.index.tolist()
pair_wins = pd.DataFrame(0, index=methods, columns=methods, dtype=int)
for i, mi in enumerate(methods):
    for j, mj in enumerate(methods):
        w = 0
        for c in use_cols:
            di, dj = df.loc[mi, c], df.loc[mj, c]
            if OBJECTIVE[c] == 1:
                if di >= dj: w += 1
            else:
                if di <= dj: w += 1
        pair_wins.iloc[i, j] = w

# ---------- 3e) Aggregate score (equal-weighted) ----------
wts = np.ones(len(use_cols)) / len(use_cols)
agg = (norm_scores.values * wts).sum(axis=1)
aggregate = pd.DataFrame({"AggregateScore": agg}, index=df.index).sort_values("AggregateScore", ascending=False)



In [4]:
import json, os
import numpy as np

# ---------- utilities ----------
def upper_quantile(values, weights, alpha):
    """
    VaR convention (upper quantile): q^+_alpha(x) = max{tau : P[X < tau] <= alpha}.
    values: (N,) sorted ascending; weights: (N,) >=0; sum_w = weights.sum() > 0
    """
    order = np.argsort(values)
    v = values[order]
    w = weights[order]
    cum_lt = np.concatenate(([0.0], np.cumsum(w[:-1]))) / w.sum()  # P[X < v_k]
    # choose largest v_k with P[X < v_k] <= alpha
    idx = np.searchsorted(cum_lt, alpha, side='right') - 1
    idx = np.clip(idx, 0, len(v)-1)
    return v[idx]

def load_mdp(domain_dir):
    P = np.load(os.path.join(domain_dir, "P.npy"))     # (S, A, S)
    R = np.load(os.path.join(domain_dir, "R.npy"))     # (S, A)
    with open(os.path.join(domain_dir, "meta.json"), "r") as f:
        meta = json.load(f)
    S, A, s0 = meta["S"], meta["A"], meta["s0"]
    assert P.shape == (S, A, S), "P must be (S,A,S)"
    assert R.shape == (S, A), "R must be (S,A)"
    return P, R, S, A, s0

# ---------- core DP: Example 4.4 & Eq. (12) ----------
def var_dp_discretized(P, R, gamma=0.9, T=100, J=4096):
    """
    Compute q^d_t(s, j, a) for t=0..T using the uniform discretization (Example 4.4).
    q[0] = 0; for t>=1, q[t] = B_u^d q[t-1].
    Returns q: list length T+1 where q[t] has shape (S, J, A).
    """
    S, A = R.shape
    # q[0] = 0 everywhere
    q_prev = np.zeros((S, J, A), dtype=np.float64)
    q_seq = [q_prev.copy()]

    # precompute uniform weights over j'
    jprime_weights = np.full(J, 1.0 / J)

    for t in range(1, T+1):
        # V_prev[s', j'] = max_a q_prev[s', j', a]
        V_prev = q_prev.max(axis=2)  # (S, J)

        # For each (s,a) we’ll build the mixture Y_{s',j'} = r(s,a)+gamma*V_prev[s',j']
        # with weight w_{s',j'} = p(s,a,s') * (1/J)
        q_curr = np.empty_like(q_prev)

        for s in range(S):
            for a in range(A):
                y = R[s, a] + gamma * V_prev  # (S, J), but R[s,a] is scalar so broadcast is fine
                # weights over s' and j'
                w_s = P[s, a]                 # (S,)
                W = (w_s[:, None]) * jprime_weights[None, :]  # (S, J)
                y_flat = y.reshape(-1)
                W_flat = W.reshape(-1)
                # for each j (risk level α=j/J), compute α-quantile of this discrete mixture
                # special-case j=0: VaR_0 is the minimum of support
                y_sorted_idx = np.argsort(y_flat)
                y_sorted = y_flat[y_sorted_idx]
                W_sorted = W_flat[y_sorted_idx]
                cum_lt = np.concatenate(([0.0], np.cumsum(W_sorted[:-1]))) / W_sorted.sum()

                for j in range(J):
                    alpha = j / J
                    if j == 0:
                        q_curr[s, j, a] = y_sorted[0]  # min support (safe, finite surrogate)
                    else:
                        # find largest v with P[X < v] <= alpha
                        idx = np.searchsorted(cum_lt, alpha, side='right') - 1
                        idx = np.clip(idx, 0, len(y_sorted)-1)
                        q_curr[s, j, a] = y_sorted[idx]

        q_seq.append(q_curr)
        q_prev = q_curr

    return q_seq  # q_seq[t] is (S,J,A)

# ---------- read out lower/upper bounds at a given α ----------
def q_lower_at_alpha(q_T, alpha):
    # q^d(s, alpha, a) = q^d(s, floor(alpha*J), a)
    S, J, A = q_T.shape
    j = int(np.floor(alpha * J))
    j = np.clip(j, 0, J-1)
    return q_T[:, j, :]  # (S, A)

def q_upper_at_alpha(q_T, alpha):
    # \bar q^d(s, alpha, a) = q^d(s, ceil(alpha*J), a)
    S, J, A = q_T.shape
    j = int(np.ceil(alpha * J))
    j = np.clip(j, 0, J-1)
    return q_T[:, j, :]  # (S, A)

# ---------- driver to compute the q̄^d row for all domains ----------
def compute_qbar_row(domains, data_root, gamma=0.9, T=100, J=4096, alpha0=0.25):
    """
    domains: ordered list of domain folder names under data_root
             (e.g., ["CW","INV1","INV2","MR","POP","RS","GR"])
    Returns dict domain->value of max_a \bar q^d_T(s0, alpha0, a)
    """
    out = {}
    for name in domains:
        P, R, S, A, s0 = load_mdp(os.path.join(data_root, name))
        q_seq = var_dp_discretized(P, R, gamma=gamma, T=T, J=J)
        qT = q_seq[-1]           # (S, J, A)
        qbar = q_upper_at_alpha(qT, alpha0)  # (S, A)
        out[name] = float(qbar[s0].max())
    return out

# ---------- example usage ----------
if __name__ == "__main__":
    # point this to your local clone: .../DRA-Q-LA/data/tabular/
    data_root = "/path/to/DRA-Q-LA/data/tabular"
    domains = ["CW","INV1","INV2","MR","POP","RS","GR"]
    row = compute_qbar_row(domains, data_root, gamma=0.9, T=100, J=4096, alpha0=0.25)
    # pretty print in the paper's order
    print("q̄^d row (α0=0.25):")
    print(" ".join(domains))
    print(" ".join(f"{row[d]:.2f}" for d in domains))


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/DRA-Q-LA/data/tabular\\CW\\P.npy'

In [5]:
# qbar_d_from_paper_data.py
# Uses the datasets from: https://github.com/MonkieDein/DRA-Q-LA (PMLR 258 Software)
# Reproduces q̄ᵈ(α0=0.25) for domains: CW, INV1, INV2, MR, POP, RS, GR.

import io, json, urllib.request, numpy as np, pandas as pd

RAW_BASE = "https://raw.githubusercontent.com/MonkieDein/DRA-Q-LA/main/experiment/domain"
DOMAINS = ["CW","INV1","INV2","MR","POP","RS","GR"]

# Paper config (AISTATS’25): gamma=0.9, T=100; Table 2 gives s0 per domain.
GAMMA = 0.9
T = 100
ALPHA0 = 0.25
S0_BY_DOMAIN = {"MR":1,"GR":5,"INV1":10,"INV2":20,"RS":9,"POP":44,"CW":37}  # Table 2

def fetch_csv(path):
    with urllib.request.urlopen(path) as r:
        return pd.read_csv(io.BytesIO(r.read()))

def load_domain(domain):
    # CSV layout used by the repo:
    #   transitions: columns [s,a,s_next,prob]
    #   rewards:     columns [s,a,reward]
    trans = fetch_csv(f"{RAW_BASE}/csv/{domain}_transition.csv")
    rews  = fetch_csv(f"{RAW_BASE}/csv/{domain}_reward.csv")
    S = int(max(trans["s"].max(), trans["s_next"].max()) + 1)
    A = int(max(trans["a"].max(), rews["a"].max()) + 1)

    P = np.zeros((S,A,S), float)
    for _, row in trans.iterrows():
        P[int(row.s), int(row.a), int(row.s_next)] += float(row.prob)
    # normalize rows (defensive)
    P = P / np.clip(P.sum(axis=2, keepdims=True), 1e-12, None)

    R = np.zeros((S,A), float)
    for _, row in rews.iterrows():
        R[int(row.s), int(row.a)] = float(row.reward)
    return P, R

def var_dp_discretized(P, R, gamma=GAMMA, T=T, J=4096):
    """Uniform j' ~ Unif({0..J-1}); Bellman operator Bu^d from Example 4.4/Eq.(12).
       q[0]=0; for t≥1, q[t]=B_u^d q[t-1].  Return q_T with shape (S,J,A)."""
    S,A = R.shape
    q_prev = np.zeros((S,J,A), float)
    j_w = np.full(J, 1.0/J)

    for _ in range(1, T+1):
        V_prev = q_prev.max(axis=2)  # (S,J)
        q_curr = np.empty_like(q_prev)
        for s in range(S):
            for a in range(A):
                # mixture over s' and j'
                y = R[s,a] + gamma * V_prev              # (S,J)
                w = P[s,a][:,None] * j_w[None,:]         # (S,J)
                y_flat = y.reshape(-1)
                w_flat = w.reshape(-1)
                order = np.argsort(y_flat)
                y_sorted = y_flat[order]
                w_sorted = w_flat[order]
                cum_lt = np.concatenate(([0.0], np.cumsum(w_sorted[:-1]))) / w_sorted.sum()
                for j in range(J):
                    if j == 0:
                        q_curr[s,j,a] = y_sorted[0]
                    else:
                        alpha = j / J
                        idx = np.searchsorted(cum_lt, alpha, side="right") - 1
                        idx = int(np.clip(idx, 0, len(y_sorted)-1))
                        q_curr[s,j,a] = y_sorted[idx]
        q_prev = q_curr
    return q_prev  # (S,J,A) at horizon T

def q_upper_at_alpha(q_T, alpha):
    S,J,A = q_T.shape
    j = min(int(np.ceil(alpha*J)), J-1)
    return q_T[:, j, :]  # (S,A)

def compute_qbar_row():
    values = {}
    for dom in DOMAINS:
        P, R = load_domain(dom)
        qT = var_dp_discretized(P, R, gamma=GAMMA, T=T, J=4096)
        qbar = q_upper_at_alpha(qT, ALPHA0)          # (S,A)
        s0 = S0_BY_DOMAIN[dom]
        values[dom] = float(qbar[s0].max())          # v̄ᵈ_T(s0,α0) = max_a q̄ᵈ_T(s0,α0,a)
        print(dom, values[dom])
    return values

if __name__ == "__main__":
    row = compute_qbar_row()
    order = ["CW","INV1","INV2","MR","POP","RS","GR"]
    print("q̄^d, α0=0.25")
    print(" ".join(order))
    print(" ".join(f"{row[d]:.2f}" for d in order))


HTTPError: HTTP Error 404: Not Found

In [6]:
# qbar_d_from_repo.py
# Uses the paper’s CSVs from:
#   https://github.com/MonkieDein/DRA-Q-LA/tree/main/experiment/domain/csv
# and domains_info.csv from:
#   https://github.com/MonkieDein/DRA-Q-LA/tree/main/experiment/domain

import io, json, urllib.request, numpy as np, pandas as pd

GITHUB_RAW = "https://raw.githubusercontent.com/MonkieDein/DRA-Q-LA/main"
CSV_BASE   = f"{GITHUB_RAW}/experiment/domain/csv"
INFO_CSV   = f"{GITHUB_RAW}/experiment/domain/domains_info.csv"

DOMAINS = ["CW","INV1","INV2","MR","POP","RS","GR"]
ALPHA0  = 0.25
T       = 100   # per paper
GAMMA   = 0.9   # per paper

def fetch_csv(url):
    with urllib.request.urlopen(url) as r:
        return pd.read_csv(io.BytesIO(r.read()))

def try_fetch_domain_csv(domain, kind):
    """
    kind in {"transition","reward"}; try singular/plural filenames.
    Returns pandas.DataFrame.
    """
    names = {
        "transition": [f"{domain}_transition.csv", f"{domain}_transitions.csv"],
        "reward":     [f"{domain}_reward.csv",     f"{domain}_rewards.csv"],
    }[kind]
    last_err = None
    for fname in names:
        url = f"{CSV_BASE}/{fname}"
        try:
            return fetch_csv(url)
        except Exception as e:
            last_err = (url, e)
    raise RuntimeError(f"Could not fetch {kind} CSV for {domain}. Tried: {names}. Last: {last_err[0]} -> {last_err[1]}")

def load_domain(domain):
    trans = try_fetch_domain_csv(domain, "transition")  # expects cols: s,a,s_next,prob
    rews  = try_fetch_domain_csv(domain, "reward")      # expects cols: s,a,reward
    S = int(max(trans["s"].max(), trans["s_next"].max()) + 1)
    A = int(max(trans["a"].max(), rews["a"].max()) + 1)
    P = np.zeros((S,A,S), float)
    for _, row in trans.iterrows():
        P[int(row.s), int(row.a), int(row.s_next)] += float(row.prob)
    # normalize defensively
    P = P / np.clip(P.sum(axis=2, keepdims=True), 1e-12, None)
    R = np.zeros((S,A), float)
    for _, row in rews.iterrows():
        R[int(row.s), int(row.a)] = float(row.reward)
    return P, R

def load_domains_info():
    info = fetch_csv(INFO_CSV)  # columns: domain, gamma, initial_state (per repo docs)
    # Build dicts with fallbacks to paper defaults
    gamma_by = {}
    s0_by    = {}
    for _, row in info.iterrows():
        name = str(row["domain"]).strip()
        gamma_by[name] = float(row.get("gamma", GAMMA))
        s0_by[name]    = int(row.get("initial_state", 0))
    return gamma_by, s0_by

# ---------- DP: Uniform-discretized VaR operator (Example 4.4 / Eq. (12)) ----------
def var_dp_discretized(P, R, gamma=GAMMA, T=T, J=4096):
    """
    q[0]=0; for t>=1: q[t] = B_u^d q[t-1] where j' ~ Uniform({0..J-1}).
    Return q_T with shape (S, J, A).
    """
    S, A = R.shape
    q_prev = np.zeros((S, J, A), float)
    j_w = np.full(J, 1.0/J)
    for _ in range(1, T+1):
        V_prev = q_prev.max(axis=2)  # (S,J)
        q_curr = np.empty_like(q_prev)
        for s in range(S):
            for a in range(A):
                y = R[s,a] + gamma * V_prev          # (S,J)
                w = P[s,a][:,None] * j_w[None,:]     # (S,J)
                y_flat = y.reshape(-1)
                w_flat = w.reshape(-1)
                order = np.argsort(y_flat)
                y_sorted = y_flat[order]
                w_sorted = w_flat[order]
                # cumulative probability of strict less-than
                cum_lt = np.concatenate(([0.0], np.cumsum(w_sorted[:-1]))) / w_sorted.sum()
                for j in range(J):
                    if j == 0:
                        q_curr[s,j,a] = y_sorted[0]  # min support
                    else:
                        alpha = j / J
                        idx = np.searchsorted(cum_lt, alpha, side="right") - 1
                        idx = int(np.clip(idx, 0, len(y_sorted)-1))
                        q_curr[s,j,a] = y_sorted[idx]
        q_prev = q_curr
    return q_prev

def q_upper_at_alpha(q_T, alpha):
    S, J, A = q_T.shape
    j = min(int(np.ceil(alpha * J)), J-1)
    return q_T[:, j, :]  # (S,A)

def compute_qbar_row():
    gamma_by, s0_by = load_domains_info()
    out = {}
    for dom in DOMAINS:
        P, R = load_domain(dom)
        gam = float(gamma_by.get(dom, GAMMA))
        qT  = var_dp_discretized(P, R, gamma=gam, T=T, J=4096)
        qbar = q_upper_at_alpha(qT, ALPHA0)
        s0   = s0_by.get(dom, 0)
        out[dom] = float(qbar[s0].max())
        print(f"{dom}: {out[dom]:.2f}")
    return out

if __name__ == "__main__":
    row = compute_qbar_row()
    order = ["CW","INV1","INV2","MR","POP","RS","GR"]
    print("q̄^d (α0=0.25):")
    print(" ".join(order))
    print(" ".join(f"{row[d]:.2f}" for d in order))

        

RuntimeError: Could not fetch transition CSV for CW. Tried: ['CW_transition.csv', 'CW_transitions.csv']. Last: https://raw.githubusercontent.com/MonkieDein/DRA-Q-LA/main/experiment/domain/csv/CW_transitions.csv -> HTTP Error 404: Not Found

In [8]:
import numpy as np, pandas as pd
from pathlib import Path
import argparse, json

# --- Domains and local CSV mapping (your uploaded files) ---
DOMAIN_FILES = {
    "CW":   "data/csv/cliff.csv",
    "INV1": "data/csv/inventory1.csv",
    "INV2": "data/csv/inventory2.csv",
    "MR":   "data/csv/machine.csv",
    "POP":  "data/csv/population.csv",
    "RS":   "data/csv/riverswim.csv",
    "GR":   "data/csv/ruin.csv",
}
# Initial states (per paper Table 2)
S0_BY_DOMAIN = {"MR":1,"GR":5,"INV1":10,"INV2":20,"RS":9,"POP":44,"CW":37}

def load_domain_from_single_csv(path):
    """
    Expect combined schema:
    idstatefrom, idaction, idstateto, probability, reward
    (exactly what the user uploaded)
    """
    df = pd.read_csv(path)
    req = {"idstatefrom","idaction","idstateto","probability","reward"}
    if not req.issubset(df.columns):
        raise ValueError(f"CSV {path} missing columns; found {df.columns.tolist()}")

    S = int(max(df["idstatefrom"].max(), df["idstateto"].max()) + 1)
    A = int(df["idaction"].max() + 1)  # assumes contiguous 0..A-1

    P = np.zeros((S, A, S), float)
    R = np.zeros((S, A), float)

    # Fill transitions and average reward per (s,a) if repeated rows
    # Group by (s,a); transitions sum to 1 across s'
    for (s,a), g in df.groupby(["idstatefrom","idaction"]):
        s = int(s); a = int(a)
        # transitions
        probs = g.groupby("idstateto")["probability"].sum()
        for s_next, prob in probs.items():
            P[s, a, int(s_next)] = float(prob)
        # reward: mean of reward column for this (s,a)
        R[s, a] = float(g["reward"].mean())

    # Normalize safeguards
    row_sums = P.sum(axis=2, keepdims=True)
    row_sums[row_sums == 0.0] = 1.0
    P = P / row_sums
    return P, R

def var_dp_discretized(P, R, gamma=0.9, T=100, J=512):
    """
    Uniform-discretized VaR operator (Example 4.4 / Eq. 12 in the paper).
    q[0]=0; q[t] = B_u^d q[t-1]. Return q_T (S,J,A).
    NOTE: J can be large (e.g., 4096) but computation grows ~O(S*A*S*J + S*A*J log(S*J)).
    """
    S, A = R.shape
    q_prev = np.zeros((S, J, A), float)
    j_w = np.full(J, 1.0/J)
    for _ in range(1, T+1):
        V_prev = q_prev.max(axis=2)  # (S,J)
        q_curr = np.empty_like(q_prev)
        for s in range(S):
            for a in range(A):
                y = R[s,a] + gamma * V_prev          # (S,J)
                w = P[s,a][:,None] * j_w[None,:]     # (S,J)
                y_flat = y.reshape(-1)
                w_flat = w.reshape(-1)
                order = np.argsort(y_flat)
                y_sorted = y_flat[order]
                w_sorted = w_flat[order]
                tot = w_sorted.sum()
                if tot <= 0:
                    # No mass -> degenerate at immediate reward
                    q_curr[s, :, a] = R[s, a]
                else:
                    cum_lt = np.concatenate(([0.0], np.cumsum(w_sorted[:-1]))) / tot
                    alphas = np.arange(J) / J
                    idxs = np.searchsorted(cum_lt, alphas, side="right") - 1
                    idxs = np.clip(idxs, 0, len(y_sorted)-1)
                    q_curr[s, :, a] = y_sorted[idxs]
        q_prev = q_curr
    return q_prev

def q_upper_at_alpha(q_T, alpha):
    S,J,A = q_T.shape
    j = min(int(np.ceil(alpha*J)), J-1)
    return q_T[:, j, :]  # (S,A)

def compute_qbar_row(domains, J=512, gamma=0.9, T=100, alpha0=0.25):
    out = {}
    for dom in domains:
        csv_path = DOMAIN_FILES[dom]
        P, R = load_domain_from_single_csv(csv_path)
        qT = var_dp_discretized(P, R, gamma=gamma, T=T, J=J)
        qbar = q_upper_at_alpha(qT, alpha0)     # (S,A)
        s0 = S0_BY_DOMAIN[dom]
        out[dom] = float(qbar[s0].max())
        print(f"{dom}: {out[dom]:.4f}")
    return out

def main():
    import argparse, json
    ap = argparse.ArgumentParser()
    ap.add_argument("--J", type=int, default=32, help="risk grid size; use 4096 for paper-level fidelity")
    ap.add_argument("--gamma", type=float, default=0.9)
    ap.add_argument("--T", type=int, default=100)
    ap.add_argument("--alpha", type=float, default=0.25)
    ap.add_argument("--out", type=str, default="qbar_row.json")
    args = ap.parse_args()

    domains = ["CW","INV1","INV2","MR","POP","RS","GR"]
    row = compute_qbar_row(domains, J=args.J, gamma=args.gamma, T=args.T, alpha0=args.alpha)
    with open(args.out, "w") as f:
        json.dump(row, f, indent=2)
    print("Saved:", args.out)

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--J J] [--gamma GAMMA] [--T T] [--alpha ALPHA] [--out OUT]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Khalid\AppData\Roaming\jupyter\runtime\kernel-08b03dfa-005b-418f-a294-56349476e8d9.json


SystemExit: 2